#PySpark Pivot and Unpivot DataFrame


---


**PySpark pivot() function is used to rotate/transpose the data from one column into multiple Dataframe columns and back using unpivot(). Pivot() It is an aggregation where one of the grouping columns values is transposed into individual columns with distinct data.**

**This tutorial describes and provides a PySpark example on how to create a Pivot table on DataFrame and Unpivot back.**


---


- Pivot PySpark DataFrame

- Pivot Performance improvement in PySpark 2.0

- Unpivot PySpark DataFrame

- Pivot or Transpose without aggregation


---


####Let’s create a PySpark DataFrame to work with.

In [0]:
from pyspark.sql.functions import expr

data = [
    ("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")
]


columns = ["Product", "Amount", "Country"]

df = spark.createDataFrame(data=data, schema=columns)

df.printSchema()
df.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



##Pivot PySpark DataFrame

---


**PySpark SQL provides pivot() function to rotate the data from one column into multiple columns. It is an aggregation where one of the grouping columns values is transposed into individual columns with distinct data. To get the total amount exported to each country of each product, will do group by Product, pivot by Country, and the sum of Amount.**

In [0]:
pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.printSchema()
pivotDF.show(truncate=False)

#This will transpose the countries from DataFrame rows into columns and produces the below output. where ever data is not present, it represents as null by default.

root
 |-- Product: string (nullable = true)
 |-- Canada: long (nullable = true)
 |-- China: long (nullable = true)
 |-- Mexico: long (nullable = true)
 |-- USA: long (nullable = true)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |2000  |1600|
|Banana |2000  |400  |null  |1000|
|Carrots|2000  |1200 |null  |1500|
+-------+------+-----+------+----+



##Pivot Performance improvement in PySpark 2.0


**version 2.0 on-wards performance has been improved on Pivot, however, if you are using the lower version; note that pivot is a very expensive operation hence, it is recommended to provide column data (if known) as an argument to function as shown below.**

In [0]:
countries = ["USA", "China", "Canada", "Mexico"]
pivotDF = df.groupBy("Product").pivot("country", countries).sum("Amount")

pivotDF.show(truncate=False)

+-------+----+-----+------+------+
|Product|USA |China|Canada|Mexico|
+-------+----+-----+------+------+
|Orange |4000|4000 |null  |null  |
|Beans  |1600|1500 |null  |2000  |
|Banana |1000|400  |2000  |null  |
|Carrots|1500|1200 |2000  |null  |
+-------+----+-----+------+------+



**Another approach is to do two-phase aggregation. PySpark 2.0 uses this implementation in order to improve the performance Spark-13749**

In [0]:
pivotDF = df.groupBy("Product", "Country")\
.sum("Amount")\
.groupBy("Product")\
.pivot("Country")\
.sum("sum(Amount)")

pivotDF.show(truncate=False)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |2000  |1600|
|Banana |2000  |400  |null  |1000|
|Carrots|2000  |1200 |null  |1500|
+-------+------+-----+------+----+



**The above two examples return the same output but with better performance.**

##Unpivot PySpark DataFrame

---


**Unpivot is a reverse operation, we can achieve by rotating column values into rows values. PySpark SQL doesn’t have unpivot function hence will use the stack() function. Below code converts column countries to row.**

In [0]:
unpivotExpr = "stack(4, 'Canada', Canada, 'China', China, 'Mexico', Mexico, 'USA', USA) as (Country, Total)"

unpivotDF = pivotDF.select("Product", expr(unpivotExpr))\
.where(" Total is not null")

unpivotDF.show()

+-------+-------+-----+
|Product|Country|Total|
+-------+-------+-----+
| Orange|  China| 4000|
| Orange|    USA| 4000|
|  Beans|  China| 1500|
|  Beans| Mexico| 2000|
|  Beans|    USA| 1600|
| Banana| Canada| 2000|
| Banana|  China|  400|
| Banana|    USA| 1000|
|Carrots| Canada| 2000|
|Carrots|  China| 1200|
|Carrots|    USA| 1500|
+-------+-------+-----+



In [0]:
df = unpivotDF.toPandas()

df = df.transpose()
df.columns = df.iloc[0]
df[1:]

Product,Orange,Orange,Beans,Beans,Beans,Banana,Banana,Banana,Carrots,Carrots,Carrots
Country,China,USA,China,Mexico,USA,Canada,China,USA,Canada,China,USA
Total,4000,4000,1500,2000,1600,2000,400,1000,2000,1200,1500
